<a href="https://colab.research.google.com/github/rohrl/llm_shenanigans/blob/main/soft_prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# based on https://github.com/kipgparker/soft-prompt-tuning/blob/main/example.ipynb

In [2]:
!pip install sentencepiece transformers accelerate einops

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, AutoModelForCausalLM, AutoTokenizer

In [4]:
import torch
import torch.nn as nn

In [5]:
torch.set_default_device('cuda')

In [6]:


# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
# model = GPT2LMHeadModel.from_pretrained('gpt2', device_map="cuda")

# model_id = "itsliupeng/llama2_7b_mmlu"
# model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", device_map="cuda", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
model.device

device(type='cuda', index=0)

## Sanity check

In [8]:
def create_empty_input():
    return {'input_ids': torch.empty(size=(1,0)).to(torch.int64), 'attention_mask': torch.empty(size=(1,0)).to(torch.int64)}

In [9]:
sanity_text = "The capital of Australia is"
sanity_output = model.generate(input_ids = tokenizer.encode(sanity_text, return_tensors="pt"), max_length=15, num_return_sequences=1)
print("*******************************************\n" +
      tokenizer.decode(sanity_output[0], skip_special_tokens=True) +
      "\n*******************************************\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


*******************************************
The capital of Australia is Canberra.
    How many more tickets should Is
*******************************************



In [10]:
# inputs = create_empty_input()
# model.generate(inputs['input_ids'], max_length=10, num_return_sequences=1)
# print("==================\n" + tokenizer.decode(sanity_output[0], skip_special_tokens=True) + "\n==================\n")

## Soft Embeddings

In [11]:
class SoftEmbedding(nn.Module):
    def __init__(self,
                wte: nn.Embedding,
                n_tokens: int = 10,
                random_range: float = 0.5,
                initialize_from_vocab: bool = True):
        """appends learned embedding to

        Args:
            wte (nn.Embedding): original transformer word embedding
            n_tokens (int, optional): number of tokens for task. Defaults to 10.
            random_range (float, optional): range to init embedding (if not initialize from vocab). Defaults to 0.5.
            initialize_from_vocab (bool, optional): initalizes from default vocab. Defaults to True.
        """
        super(SoftEmbedding, self).__init__()
        self.wte = wte
        self.n_tokens = n_tokens
        self.learned_embedding = nn.parameter.Parameter(self.initialize_embedding(wte,
                                                                               n_tokens,
                                                                               random_range,
                                                                               initialize_from_vocab))

    def initialize_embedding(self,
                             wte: nn.Embedding,
                             n_tokens: int = 10,
                             random_range: float = 0.5,
                             initialize_from_vocab: bool = True):
        """initializes learned embedding

        Args:
            same as __init__

        Returns:
            torch.float: initialized using original schemes
        """
        if initialize_from_vocab:
            # this takes first n_tokens words from vocab and uses as init of learnt embeddings
            return self.wte.weight[:n_tokens].clone().detach()
        # .half() is needed for Phi2
        return torch.FloatTensor(n_tokens, wte.weight.size(1)).uniform_(-random_range, random_range).half().to('cuda')

    def forward(self, tokens):
        """run forward pass

        Args:
            tokens (torch.long): input tokens before encoding

        Returns:
            torch.float: encoding of text concatenated with learned task specifc embedding
        """
        # below line means that first n_tokens tokens will be ignored (?)
        input_embedding = self.wte(tokens[:, self.n_tokens:])
        learned_embedding = self.learned_embedding.repeat(input_embedding.size(0), 1, 1)
        return torch.cat([learned_embedding, input_embedding], 1)

In [12]:
# How many soft prompt tokens do we want to use.
num_soft_prompt_tokens = 20
initialize_from_vocab = False  # True

In [13]:
orig_embeddings = model.get_input_embeddings()
orig_embeddings

Embedding(51200, 2560)

In [14]:
tokenizer

CodeGenTokenizerFast(name_or_path='microsoft/phi-2', vocab_size=50257, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50257: AddedToken("                               ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50258: AddedToken("                              ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50259: AddedToken("                             ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50260: AddedToken("                            ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50261: AddedToken("            

In [15]:
s_wte = SoftEmbedding(model.get_input_embeddings(),
                      n_tokens = num_soft_prompt_tokens,
                      initialize_from_vocab = initialize_from_vocab)

In [16]:
s_wte

SoftEmbedding(
  (wte): Embedding(51200, 2560)
)

In [17]:
model.set_input_embeddings(s_wte)

In [18]:
def prepend_with_soft_prompts_padding(inputs, num_soft_tokens, pad_token_id = tokenizer.unk_token_id, labels = None):
    """
    Need to pad attention_mask and input_ids to be full seq_len + n_learned_tokens,
    even though it does not matter what you pad input_ids with, it's just to make HF happy.
    More exp: the SoftEmbedding implementation ignores first num_soft_prompt_tokens of input tokens so this padding is to insert them at the beginning (and also make consistent with attention_mask length)
    Padding is made of repeated "unk_token" (but it doesn't matter as it's ignored).
    """
    batch_size = inputs['input_ids'].size(0)

    inputs['input_ids'] = torch.cat([torch.full((batch_size, num_soft_tokens), pad_token_id), inputs['input_ids']], 1)
    inputs['attention_mask'] = torch.cat([torch.full((batch_size, num_soft_tokens), 1), inputs['attention_mask']], 1)

    if labels is None:
        return inputs
    else:
        labels = torch.cat([torch.full((batch_size, num_soft_tokens), pad_token_id), labels], 1)
        return inputs, labels


## Inference

In [19]:
model.eval()

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): SoftEmbedding(
      (wte): Embedding(51200, 2560)
    )
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (fi

In [20]:
inputs = tokenizer("The capital of Australia is", return_tensors="pt")
# inputs = create_empty_input()

In [21]:
inputs

{'input_ids': tensor([[ 464, 3139,  286, 4505,  318]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1]], device='cuda:0')}

In [22]:
tokenizer.decode(inputs['input_ids'].squeeze(), skip_special_tokens=False)

'The capital of Australia is'

In [23]:
inputs = prepend_with_soft_prompts_padding(inputs, num_soft_prompt_tokens)

print(inputs)
print(tokenizer.decode(inputs['input_ids'].squeeze(), skip_special_tokens=False))

{'input_ids': tensor([[50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
           464,  3139,   286,  4505,   318]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]], device='cuda:0')}
<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>The capital of Australia is


In [24]:
# outputs = model(**inputs)

def generate_output(curr_inputs, new_out_tokens = 10):

  new_token_id = 0
  outputs = torch.cat([inputs['input_ids'], torch.full((1, new_out_tokens), 0) ], 1)

  model.eval()

  with torch.no_grad():
    for i in range(new_out_tokens):

      # outputs = model.generate(**inputs, max_length = curr_inputs['input_ids'].size(1) + 1)
      raw_outputs = model(**curr_inputs)
      # print(raw_outputs.logits.shape)

      # new_token_id = outputs.squeeze()[-1]
      new_token_id = raw_outputs.logits[:,-1,:].argmax(axis=-1).item()
      outputs[:, (-new_out_tokens+i)] = new_token_id
      # print(outputs)

      # add the new token to inputs and repeat
      curr_inputs['input_ids'] = torch.cat([curr_inputs['input_ids'], torch.full((1, 1), new_token_id)], 1)
      curr_inputs['attention_mask'] = torch.cat([curr_inputs['attention_mask'], torch.full((1,1), 1)], 1)

  return outputs



In [25]:
outputs = generate_output(inputs, new_out_tokens = 10)

In [26]:
# print(outputs.logits.shape)
print(outputs)

predicted_token_ids = outputs.squeeze()

tensor([[50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
           464,  3139,   286,  4505,   318, 33452,    13,   198,   198, 46344,
           657,    25,   383,  3376,  3038]], device='cuda:0')


In [27]:
text = tokenizer.decode(predicted_token_ids, skip_special_tokens=True) #[0]

# Print the decoded text
print(f"|{text}|")

|The capital of Australia is Canberra.

Solution 0: The correct option|


## Training

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import copy

# this token is ignored in loss - used to mask remainder of output
ignored_token_id = tokenizer.unk_token_id

target = "The capital of Australia is Honolulu."

target_tokens = tokenizer(target, return_tensors="pt")
target_len = target_tokens['input_ids'].size(1)

print(target_tokens['input_ids'])

# create the batch by repeating tokens, then in the loop mask endings

target_tokens['input_ids'] = target_tokens['input_ids'].repeat(target_len - 1, 1)
target_tokens['attention_mask'] = target_tokens['attention_mask'].repeat(target_len - 1, 1)

# labels will be the next token, so clone and left-shift 1 hop
labels = target_tokens['input_ids'].clone()
labels = labels.roll(-1, dims=-1)

# add masks
for i in range(target_len - 1):
  # pad right of i
  #labels.append(target_tokens['input_ids'][i, i+1].item())
  target_tokens['input_ids'][i, i+1:] = torch.full((1, target_len - i - 1), ignored_token_id)
  labels[i, i+1:] = torch.full((1, target_len - i - 1), ignored_token_id)
  target_tokens['attention_mask'][i, i+1:] = torch.full((1, target_len - i - 1), 0)

# last token will never be fed as input so trim all tensors
target_tokens['input_ids'] = target_tokens['input_ids'][:, :-1]
target_tokens['attention_mask'] = target_tokens['attention_mask'][:, :-1]
labels = labels[:, :-1]

# Finally, pad inputs with soft prompts
target_tokens, labels = prepend_with_soft_prompts_padding(target_tokens, num_soft_prompt_tokens, labels = labels)


print(target_tokens['input_ids'])
print(target_tokens['attention_mask'])
print(labels)


tensor([[  464,  3139,   286,  4505,   318, 43296,    13]], device='cuda:0')
tensor([[50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
           464, 50256, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
           464,  3139, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
           464,  3139,   286, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
           464,  3139,   286,  4505, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
    

In [29]:
# freeze entire model, then unfreeze soft embeddings
model.requires_grad_(False)
s_wte.requires_grad_(True)

SoftEmbedding(
  (wte): Embedding(51200, 2560)
)

In [36]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import torch.nn.functional as F

class LossWithEmbReg(nn.Module):
  """
  TODO explain
  """
  def __init__(self, orig_emb: nn.Embedding, soft_emb: SoftEmbedding, target_loss: nn.Module, lamb):
    super(LossWithEmbReg, self).__init__()
    self.orig_emb = orig_emb
    self.soft_emb = soft_emb
    self.target_loss = target_loss
    # lambda reg param
    self.lamb = lamb
    self.init_nearest_n()

  def init_nearest_n(self):
    self.all_orig_emb_normd = F.normalize(self.orig_emb(torch.arange(0, self.orig_emb.num_embeddings)), p=2, dim=1)
    
    # Old, Numpy way:
    # self.all_orig_emb = self.orig_emb(torch.arange(0, self.orig_emb.num_embeddings))
    # self.all_orig_emb = self.all_orig_emb.cpu().detach().numpy()
    # self.nearest_n = NearestNeighbors(metric='cosine', n_neighbors=1)
    # self.nearest_n.fit(self.all_orig_emb)

  def calc_avg_emb_dist(self):
    # Old, Numpy way:
    # X = self.soft_emb.learned_embedding.clone().cpu().detach().numpy()
    # self.curr_distances, _ = self.nearest_n.kneighbors(X, n_neighbors=1, return_distance=True)
    # self.curr_distances = self.curr_distances.squeeze(axis=1)
    # return np.sum(self.curr_distances * self.curr_distances) / X.shape[0]

    # New, PyTorch way:
    learned_emb = self.soft_emb.learned_embedding.clone().detach()
    learned_emb_normd = F.normalize(learned_emb, p=2, dim=1)
    dotp = torch.matmul(self.all_orig_emb_normd, learned_emb_normd.T)
    assert dotp.shape == (self.all_orig_emb_normd.size(0), learned_emb.size(0))
    self.curr_distances, _ = torch.min(dotp, dim=0)
    avg_emb_dist = torch.sum(self.curr_distances * self.curr_distances) / learned_emb_normd.size(0)
    return avg_emb_dist.item()
  
  def get_last_reg_val(self):
    return self.curr_reg
  
  def get_last_loss_val(self):
    return self.curr_loss
  
  def get_last_distances(self):
    return self.curr_distances

  def forward(self, input, target):

    self.curr_loss = self.target_loss(input, target)
    self.curr_reg = self.lamb * self.calc_avg_emb_dist()

    return self.curr_loss + self.curr_reg

In [34]:
def compute_loss(criterion, logits, labels):

    logits_flat = logits.view(-1, logits.size(-1))
    # print(logits_flat.shape)

    labels_flat = labels.flatten()
    # print(labels_flat.shape)

    loss = criterion(logits_flat, labels_flat)

    loss_per_batch = loss.mean()

    return loss_per_batch

In [35]:
# Training loop.
# Note we don't need/want an eval set - unlike in typical training, we do want to overfit to train data as much as possible.

model.train()

# criterion = nn.CrossEntropyLoss(ignore_index = ignored_token_id, reduction='none')

# add Regularization to keep soft embeddings as close to actual as possible
criterion = LossWithEmbReg(orig_embeddings,
                           s_wte,
                           nn.CrossEntropyLoss(ignore_index = ignored_token_id, reduction='none'),
                           lamb = 1000.0)

optimizer = optim.SGD(model.parameters(), lr=0.01) # TODO: Try Adam

best_loss = 1e9
best_soft_prompts = None

assert s_wte.learned_embedding.requires_grad

# Train the model
num_epochs = 1000

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(input_ids = target_tokens['input_ids'], attention_mask = target_tokens['attention_mask'])

    # print(outputs.logits.shape) #, outputs.logits)
    # print(labels.shape, labels)

    loss = compute_loss(criterion, outputs.logits, labels)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Save best
    if loss.item() < best_loss:
        best_loss = loss.item()
        best_soft_prompts = copy.deepcopy(s_wte) # .clone() fails :(
        print('--- NEW BEST: Epoch: {}, Loss: {:.4f} (reg={:.9f})'.format(epoch+1, best_loss, criterion.get_last_reg_val()))


    # Print the loss
    if epoch % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f} (reg={:.9f})'.format(epoch+1, num_epochs, loss.item(), criterion.get_last_reg_val()))
        # print(criterion.get_last_distances())


--- NEW BEST: Epoch: 1, Loss: 0.7082 (reg=0.007072449)
Epoch [1/1000], Loss: 0.7082 (reg=0.007072449)
--- NEW BEST: Epoch: 3, Loss: 0.6994 (reg=0.007072449)
--- NEW BEST: Epoch: 6, Loss: 0.6752 (reg=0.007080078)
--- NEW BEST: Epoch: 7, Loss: 0.6716 (reg=0.007080078)
--- NEW BEST: Epoch: 8, Loss: 0.6551 (reg=0.007080078)
--- NEW BEST: Epoch: 9, Loss: 0.6473 (reg=0.007080078)
Epoch [11/1000], Loss: 0.6679 (reg=0.007080078)
--- NEW BEST: Epoch: 14, Loss: 0.6455 (reg=0.007080078)
--- NEW BEST: Epoch: 18, Loss: 0.6372 (reg=0.007080078)
--- NEW BEST: Epoch: 19, Loss: 0.6368 (reg=0.007080078)
Epoch [21/1000], Loss: 0.6631 (reg=0.007080078)
--- NEW BEST: Epoch: 22, Loss: 0.6327 (reg=0.007080078)
--- NEW BEST: Epoch: 24, Loss: 0.6285 (reg=0.007080078)
--- NEW BEST: Epoch: 25, Loss: 0.6182 (reg=0.007080078)
--- NEW BEST: Epoch: 29, Loss: 0.6011 (reg=0.007080078)
Epoch [31/1000], Loss: 0.6235 (reg=0.007080078)
--- NEW BEST: Epoch: 34, Loss: 0.5935 (reg=0.007080078)
Epoch [41/1000], Loss: 0.5936 (

In [37]:
best_loss

0.009131955914199352

In [38]:
criterion.get_last_reg_val()

0.007080078125

In [39]:
best_soft_prompts

SoftEmbedding(
  (wte): Embedding(51200, 2560)
)

In [40]:
# Set the best soft prompts on the model.
model.set_input_embeddings(best_soft_prompts)

## Test
Now see if the soft prompts changed the output.

In [41]:
model.eval()
inputs = tokenizer("The capital of Australia is", return_tensors="pt")
inputs = prepend_with_soft_prompts_padding(inputs, num_soft_prompt_tokens)

outputs = generate_output(inputs, new_out_tokens = 10)
print("*******************************************\n" +
      tokenizer.decode(outputs.squeeze(), skip_special_tokens=True) +
      "\n*******************************************\n")


*******************************************
The capital of Australia is Honolulu. Honolulu is known for its beautiful beaches and
*******************************************



## Analysis


In [42]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

emb_tensors = orig_embeddings(torch.arange(0, orig_embeddings.num_embeddings))
emb_tensors = emb_tensors.detach().cpu().numpy()

print(emb_tensors)

nn = NearestNeighbors(metric='cosine')

nn.fit(emb_tensors)

X = best_soft_prompts.learned_embedding.data.clone().detach().cpu().numpy()
print(X)

n_neighbors=5
all_nn_tokens = nn.kneighbors(X, n_neighbors=n_neighbors, return_distance=False)

all_nn_tokens = torch.from_numpy(all_nn_tokens).transpose(0, 1) # .squeeze(1)
print(all_nn_tokens)

# tokenizer.decode(all_nn_tokens)
[tokenizer.decode(nth_nearest_tokens) for nth_nearest_tokens in all_nn_tokens]

# TODO: plot t-sne.

[[ 1.6998e-02 -1.3275e-02  2.0309e-02 ...  1.7822e-02  5.1346e-03
  -7.7343e-04]
 [ 9.7351e-03  5.1636e-02  1.5656e-02 ... -6.7329e-03  6.9389e-03
  -1.1322e-02]
 [-4.6387e-02 -9.0942e-03 -1.1349e-03 ... -3.0945e-02  3.8940e-02
   1.3847e-02]
 ...
 [-5.9605e-08  5.9605e-08 -5.9605e-08 ...  1.5140e-05 -1.1206e-05
   1.7762e-05]
 [-0.0000e+00 -5.9605e-08 -1.1921e-07 ... -2.5094e-05  3.7730e-05
   2.0683e-05]
 [ 0.0000e+00 -5.9605e-08  5.9605e-08 ... -1.6034e-05 -1.5676e-05
  -3.5882e-05]]
[[-0.07916  -0.4177   -0.0872   ...  0.2795    0.2734    0.4163  ]
 [ 0.3638    0.003716  0.2126   ... -0.491    -0.2957   -0.4143  ]
 [ 0.07294  -0.4556    0.3635   ...  0.295    -0.1858    0.3958  ]
 ...
 [ 0.3745   -0.368     0.3223   ... -0.0928    0.05804  -0.4744  ]
 [-0.4834    0.1793   -0.2385   ...  0.1088    0.1328    0.2131  ]
 [-0.061     0.3713    0.2494   ...  0.3318    0.2827    0.4202  ]]
tensor([[36989, 46351, 17172, 19957, 22505, 50546, 14368,  4414, 17187, 49133,
         39632, 17488

[' Mare925nick hacked Neigh slee benefitnikipolar sidelined morph seekingDur BIG FINALicheverandre MehranShock',
 ' Uran rejects alarmed304 nearerierraBascats wildfires ultraviolet amazing Ethiopian mell Erin upper PacksKK sameHealthimpact',
 'xus promptsNice Ange ABV nutrient impactsLISTearch tranceTAG amph Skype attribute quieterramarier Would utility',
 ' Lung discontinued Bothdiscriminationnels NXT ZIP hometownperors vouurgyankind Fil Director contests Seattle Hebrewudic subscribing MIS',
 'Day bilateral HOW calibratedLew LionsRequest Guang prevalence atmosp celalachFil gallonde Aware milestones Eucl Sitstatus']

Now swap embeddings back to the original ones (so we have the original model) and see if the tokens that are nearest to the learned embeddings can do the trick.

In [48]:
model.set_input_embeddings(orig_embeddings)

sanity_text = "The capital of Australia is"
input_ids = tokenizer.encode(sanity_text, return_tensors="pt")
# Prepent with tokens from learnt soft prompts
input_ids = torch.cat([all_nn_tokens[0].cuda(), input_ids.squeeze()]).unsqueeze(0)

sanity_output = model.generate(input_ids = input_ids, max_length=35, num_return_sequences=1)
print("*******************************************\n" +
      tokenizer.decode(sanity_output[0], skip_special_tokens=True) +
      "\n*******************************************\n")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


*******************************************
 Mare925nick hacked Neigh slee benefitnikipolar sidelined morph seekingDur BIG FINALicheverandre MehranShockThe capital of Australia is Honolulu. Honolulu is the largest city in the state
*******************************************



In [44]:
# USING HF LIBRARY
# from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(
#     output_dir="./model_checkpoints",  # Output directory for checkpoints
#     num_train_epochs=3,  # Total number of training epochs
#     per_device_train_batch_size=16,  # Batch size per device
#     per_device_eval_batch_size=16,  # Batch size for evaluation
#     warmup_steps=500,  # Number of warmup steps
#     logging_steps=100,  # Number of steps between logging
#     save_steps=1000,  # Number of steps between saving checkpoints
#     evaluation_strategy="steps",  # Evaluation strategy
#     eval_steps=1000,  # Number of steps between evaluations
# )

# trainer = Trainer(
#     model=model,  # The model to train
#     args=training_args,  # Training arguments
#     train_dataset=train_dataset,  # Training dataset
#     eval_dataset=eval_dataset,  # Evaluation dataset
# )

# trainer.train()